In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -U autogluon > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

from autogluon.multimodal import MultiModalPredictor

print("✅ Cài đặt và import thư viện hoàn tất.")

2025-07-01 03:29:00.423881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751340540.653151      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751340540.717745      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ Cài đặt và import thư viện hoàn tất.


In [4]:
BASE_PATH = '/kaggle/input/petfinder-adoption-prediction'
TRAIN_PATH = os.path.join(BASE_PATH, 'train/train.csv')
BREED_PATH = os.path.join(BASE_PATH, 'breed_labels.csv')
COLOR_PATH = os.path.join(BASE_PATH, 'color_labels.csv')
STATE_PATH = os.path.join(BASE_PATH, 'state_labels.csv')
IMAGE_PATH = os.path.join(BASE_PATH, 'train_images/')
train_df = pd.read_csv(TRAIN_PATH)
breed_df = pd.read_csv(BREED_PATH)
color_df = pd.read_csv(COLOR_PATH)
state_df = pd.read_csv(STATE_PATH)

In [5]:
train_df = train_df.merge(breed_df.rename(columns={'BreedID': 'Breed1', 'BreedName': 'Breed1Name'}), on='Breed1', how='left')
train_df = train_df.merge(breed_df.rename(columns={'BreedID': 'Breed2', 'BreedName': 'Breed2Name'}), on='Breed2', how='left')
train_df = train_df.merge(color_df.rename(columns={'ColorID': 'Color1', 'ColorName': 'Color1Name'}), on='Color1', how='left')
train_df = train_df.merge(state_df.rename(columns={'StateID': 'State', 'StateName': 'StateName'}), on='State', how='left')
train_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Type_x,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Description,PetID,PhotoAmt,AdoptionSpeed,Type_y,Breed1Name,Type,Breed2Name,Color1Name,StateName
0,2,Nibble,3,299,0,1,1,7,0,1,...,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,2.0,Tabby,NaN,NaN,Black,Selangor
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,2.0,Domestic Medium Hair,NaN,NaN,Black,Kuala Lumpur
2,1,Brisco,1,307,0,1,2,7,0,2,...,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3,1.0,Mixed Breed,NaN,NaN,Brown,Selangor
3,1,Miko,4,307,0,2,1,2,0,2,...,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2,1.0,Mixed Breed,NaN,NaN,Black,Kuala Lumpur
4,1,Hunter,1,307,0,1,1,0,0,2,...,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2,1.0,Mixed Breed,NaN,NaN,Black,Selangor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,2,NaN,2,266,0,3,1,0,0,2,...,I have 4 kittens that need to be adopt urgentl...,dc0935a84,3.0,2,2.0,Domestic Short Hair,NaN,NaN,Black,Selangor
14989,2,Serato & Eddie,60,265,264,3,1,4,7,2,...,Serato(female cat- 3 color) is 4 years old and...,a01ab5b30,3.0,4,2.0,Domestic Medium Hair,2.0,Domestic Long Hair,Black,Selangor
14990,2,Monkies,2,265,266,3,5,6,7,3,...,"Mix breed, good temperament kittens. Love huma...",d981b6395,5.0,3,2.0,Domestic Medium Hair,2.0,Domestic Short Hair,Cream,Selangor
14991,2,Ms Daym,9,266,0,2,4,7,0,1,...,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,4,2.0,Domestic Short Hair,NaN,NaN,Yellow,Johor


In [6]:
train_df['image'] = train_df['PetID'].apply(lambda x: os.path.join(IMAGE_PATH, f'{x}-1.jpg'))
train_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Type_x,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,PetID,PhotoAmt,AdoptionSpeed,Type_y,Breed1Name,Type,Breed2Name,Color1Name,StateName,image
0,2,Nibble,3,299,0,1,1,7,0,1,...,86e1089a3,1.0,2,2.0,Tabby,NaN,NaN,Black,Selangor,/kaggle/input/petfinder-adoption-prediction/tr...
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,6296e909a,2.0,0,2.0,Domestic Medium Hair,NaN,NaN,Black,Kuala Lumpur,/kaggle/input/petfinder-adoption-prediction/tr...
2,1,Brisco,1,307,0,1,2,7,0,2,...,3422e4906,7.0,3,1.0,Mixed Breed,NaN,NaN,Brown,Selangor,/kaggle/input/petfinder-adoption-prediction/tr...
3,1,Miko,4,307,0,2,1,2,0,2,...,5842f1ff5,8.0,2,1.0,Mixed Breed,NaN,NaN,Black,Kuala Lumpur,/kaggle/input/petfinder-adoption-prediction/tr...
4,1,Hunter,1,307,0,1,1,0,0,2,...,850a43f90,3.0,2,1.0,Mixed Breed,NaN,NaN,Black,Selangor,/kaggle/input/petfinder-adoption-prediction/tr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,2,NaN,2,266,0,3,1,0,0,2,...,dc0935a84,3.0,2,2.0,Domestic Short Hair,NaN,NaN,Black,Selangor,/kaggle/input/petfinder-adoption-prediction/tr...
14989,2,Serato & Eddie,60,265,264,3,1,4,7,2,...,a01ab5b30,3.0,4,2.0,Domestic Medium Hair,2.0,Domestic Long Hair,Black,Selangor,/kaggle/input/petfinder-adoption-prediction/tr...
14990,2,Monkies,2,265,266,3,5,6,7,3,...,d981b6395,5.0,3,2.0,Domestic Medium Hair,2.0,Domestic Short Hair,Cream,Selangor,/kaggle/input/petfinder-adoption-prediction/tr...
14991,2,Ms Daym,9,266,0,2,4,7,0,1,...,e4da1c9e4,3.0,4,2.0,Domestic Short Hair,NaN,NaN,Yellow,Johor,/kaggle/input/petfinder-adoption-prediction/tr...


In [7]:
train_df.dropna(subset=['image'], inplace=True)


In [8]:
print("✅ Dữ liệu đã được chuẩn bị xong.")
print("Xem qua dữ liệu sau khi hợp nhất và thêm đường dẫn ảnh:")
print(train_df[['PetID', 'Name', 'AdoptionSpeed', 'Breed1Name', 'image']].head())

✅ Dữ liệu đã được chuẩn bị xong.
Xem qua dữ liệu sau khi hợp nhất và thêm đường dẫn ảnh:
       PetID         Name  AdoptionSpeed            Breed1Name  \
0  86e1089a3       Nibble              2                 Tabby   
1  6296e909a  No Name Yet              0  Domestic Medium Hair   
2  3422e4906       Brisco              3           Mixed Breed   
3  5842f1ff5         Miko              2           Mixed Breed   
4  850a43f90       Hunter              2           Mixed Breed   

                                               image  
0  /kaggle/input/petfinder-adoption-prediction/tr...  
1  /kaggle/input/petfinder-adoption-prediction/tr...  
2  /kaggle/input/petfinder-adoption-prediction/tr...  
3  /kaggle/input/petfinder-adoption-prediction/tr...  
4  /kaggle/input/petfinder-adoption-prediction/tr...  


In [9]:
features_to_use = [
    'image',                                  # Ảnh
    'Description',                            # Text
    'Age', 'Type', 'Breed1Name', 'Gender',    # Bảng
    'Color1Name', 'MaturitySize', 'FurLength',
    'Vaccinated', 'Dewormed', 'Sterilized',
    'Health', 'Quantity', 'Fee', 'StateName',
    'PhotoAmt'
]

In [10]:
label_col = 'AdoptionSpeed'

In [11]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(train_df, test_size = 0.2, stratify = train_df['AdoptionSpeed'], random_state = 3)

In [12]:
predictor = MultiModalPredictor(
    label=label_col,
    problem_type='multiclass',
    eval_metric='quadratic_kappa'
)
print("\n🚀 Bắt đầu huấn luyện mô hình (có thể mất nhiều thời gian)...")
predictor.fit(
    train_data=train_data[features_to_use + [label_col]],
    time_limit=3000,
    clean_ckpts = True,
    presets='best_quality'
)

No path specified. Models will be saved in: "AutogluonModels/ag-20250701_032940"
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Pytorch Version:    2.6.0+cu124
CUDA Version:       12.4
Memory Avail:       29.40 GB / 31.35 GB (93.8%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)



🚀 Bắt đầu huấn luyện mô hình (có thể mất nhiều thời gian)...


/usr/local/lib/python3.11/dist-packages/autogluon/multimodal/data/infer_types.py:248: UserWarning: Among 500 sampled images in column 'image', 2% images can't be open. You may need to thoroughly check your data to see the percentage of missing images, and estimate the potential influence. By default, we use an image with zero pixels. You can also set hyperparameter 'data.image.missing_value_strategy' to be 'skip', which skips samples that contain a missing image.
  warnings.warn(

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /kaggle/working/AutogluonModels/ag-20250701_032940
    ```

INFO: Seed set to 0


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/autogluon/multimodal/data/infer_types.py:248: UserWarning: Among 500 sampled images in column 'image', 3% images can't be open. You may need to thoroughly check your data to see the percentage of missing images, and estimate the potential influence. By default, we use an image with zero pixels. You can also set hyperparameter 'data.image.missing_value_strategy' to be 'skip', which skips samples that contain a missing image.
  warnings.warn(
INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/autogluon/multimodal/data/infer_types.py:248: UserWarning: Among 500 sampled images in column 'image', 3% images can't be open. You may need to thoroughly check your data to see the percentage of missing images, and estimate the potential influence. By default, we use an image with zero pixels. You can also set hyperparameter 'data.image.missing_value_strategy' to be 'skip', which skips samples that contain a missing image.
  warnings.warn(
INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/kaggle/working/AutogluonModels/ag-20250701_032940")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




In [15]:
predictor.evaluate(test_data[features_to_use + [label_col]], metrics = ['accuracy', 'log_loss', 'f1_macro', 'f1_weighted', 'roc_auc_ovr', 'roc_auc_ovr_weighted'])


/usr/local/lib/python3.11/dist-packages/autogluon/multimodal/data/infer_types.py:248: UserWarning: Among 500 sampled images in column 'image', 3% images can't be open. You may need to thoroughly check your data to see the percentage of missing images, and estimate the potential influence. By default, we use an image with zero pixels. You can also set hyperparameter 'data.image.missing_value_strategy' to be 'skip', which skips samples that contain a missing image.
  warnings.warn(
INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

{'accuracy': 0.39413137712570856,
 'log_loss': 1.3602803416398057,
 'f1_macro': 0.29392164186639463,
 'f1_weighted': 0.36287625004323076,
 'roc_auc_ovr': 0.6600105450908367,
 'roc_auc_ovr_weighted': 0.6706129656402963}